In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = "http://www.xssed.com/archive/page={}/"

In [3]:
mirrors = []
for i in range(1,31):
    r = requests.get(url.format(1))
    soup = BeautifulSoup(r.content,"html5lib")

    for link in soup.find_all('a'):
        if "mirror" in link.get("href"):
            mirrors.append(link.get("href"))

In [4]:
mirrors[0]

'/mirror/81282/'

In [5]:
url = "http://www.xssed.com{}"
malicious_urls = []
mirror_urls = []

for m in mirrors:
    mirror = url.format(m)
    r = requests.get(mirror)
    soup = BeautifulSoup(r.content,"html5lib")
    malicious_urls.append(soup.select("#contentpaneOpen > table > tbody > tr:nth-child(4) > th")[0].text[5:])
    mirror_urls.append(soup.select("#contentpaneOpen > table > tbody > tr:nth-child(5) > th > a")[0].get("href"))

In [6]:
def write_file(filename,strings):
    with open(filename, "w") as file:
        for string in strings:
            file.write(string + "\n")

write_file("mirrors.txt",mirror_urls)
write_file("malicious_urls.txt",malicious_urls)

In [7]:
import re

In [8]:
end = red = yellow = ''

def dom(response):
    print(response)
    highlighted = []
    sources = r'''\b(?:document\.(URL|documentURI|URLUnencoded|baseURI|cookie|referrer)|location\.(href|search|hash|pathname)|window\.name|history\.(pushState|replaceState)(local|session)Storage)\b'''
    sinks = r'''\b(?:eval|evaluate|execCommand|assign|navigate|getResponseHeaderopen|showModalDialog|Function|set(Timeout|Interval|Immediate)|execScript|crypto.generateCRMFRequest|ScriptElement\.(src|text|textContent|innerText)|.*?\.onEventName|document\.(write|writeln)|.*?\.innerHTML|Range\.createContextualFragment|(document|window)\.location)\b'''
    scripts = re.findall(r'(?i)(?s)<script[^>]*>(.*?)</script>', response)
    sinkFound, sourceFound = False, False
    for script in scripts:
        script = script.split('\n')
        num = 1
        allControlledVariables = set()
        try:
            for newLine in script:
                line = newLine
                parts = line.split('var ')
                controlledVariables = set()
                if len(parts) > 1:
                    for part in parts:
                        for controlledVariable in allControlledVariables:
                            if controlledVariable in part:
                                controlledVariables.add(re.search(r'[a-zA-Z$_][a-zA-Z0-9$_]+', part).group().replace('$', '\$'))
                pattern = re.finditer(sources, newLine)
                for grp in pattern:
                    if grp:
                        source = newLine[grp.start():grp.end()].replace(' ', '')
                        if source:
                            if len(parts) > 1:
                               for part in parts:
                                    if source in part:
                                        controlledVariables.add(re.search(r'[a-zA-Z$_][a-zA-Z0-9$_]+', part).group().replace('$', '\$'))
                            line = line.replace(source, yellow + source + end)
                for controlledVariable in controlledVariables:
                    allControlledVariables.add(controlledVariable)
                for controlledVariable in allControlledVariables:
                    matches = list(filter(None, re.findall(r'\b%s\b' % controlledVariable, line)))
                    if matches:
                        sourceFound = True
                        line = re.sub(r'\b%s\b' % controlledVariable, yellow + controlledVariable + end, line)
                pattern = re.finditer(sinks, newLine)
                for grp in pattern:
                    if grp:
                        sink = newLine[grp.start():grp.end()].replace(' ', '')
                        if sink:
                            line = line.replace(sink, red + sink + end)
                            sinkFound = True
                if line != newLine:
                    highlighted.append('%-3s %s' % (str(num), line.lstrip(' ')))
                num += 1
        except MemoryError:
            pass
    if sinkFound or sourceFound:
        return highlighted
    else:
        return []

<>:23: SyntaxWarning: invalid escape sequence '\$'
<>:32: SyntaxWarning: invalid escape sequence '\$'
<>:23: SyntaxWarning: invalid escape sequence '\$'
<>:32: SyntaxWarning: invalid escape sequence '\$'
C:\Users\kisho\AppData\Local\Temp\ipykernel_28152\1574772474.py:23: SyntaxWarning: invalid escape sequence '\$'
  controlledVariables.add(re.search(r'[a-zA-Z$_][a-zA-Z0-9$_]+', part).group().replace('$', '\$'))
C:\Users\kisho\AppData\Local\Temp\ipykernel_28152\1574772474.py:32: SyntaxWarning: invalid escape sequence '\$'
  controlledVariables.add(re.search(r'[a-zA-Z$_][a-zA-Z0-9$_]+', part).group().replace('$', '\$'))


In [9]:
def file_to_list(filename):
    lines = []
    with open(filename,'r') as file:
        for line in file:
            cleaned_line = line.strip()
            lines.append(cleaned_line)
    return lines

mirrors = file_to_list("mirrors.txt")

In [10]:
print(mirrors[1])
print(dom(requests.get(mirrors[1]).text))

http://vuln.xssed.net/2014/01/05/store.samsung.com/
<!DOCTYPE HTML>
<html lang="en">
<head>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    
<title>NX Camera 52mm C-PL Filter  | Online Shop UK</title>
    <meta name="title" content="NX Camera 52mm C-PL Filter  | Online Shop UK" />
    <meta name="description" content="Online Shop UK NX Camera 52mm C-PL Filter LF52PL Tailor your shots and see the difference" />
        <meta name="keywords" content="Online Shop UK NX Camera 52mm C-PL Filter LF52PL Tailor your shots and see the difference" />
        <meta name="robots" content="index,follow" />
        <meta id="viewport" name="viewport" content="width=500, user-scalable=yes, target-densitydpi=device-dpi" />
    <meta name="HandheldFriendly" content="false" />
    <meta name="MobileOptimized" content="750" />
<meta property="fb:app_id" content="148976658633899" />
    <meta property="og:title" cont

In [11]:
for i in range(10,100):
    print(dom(requests.get(mirrors[i]).text))

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns:fb="http://www.facebook.com/2008/fbml">
	<head>
				<meta name="viewport" content="width=device-width; initial-scale=1.0; maximum-scale=1.0; user-scalable=0;" />
		<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1" />
		<meta name="PageType" content="7" />
		<meta name="PageID" content="38" />
		<meta name="last-modified" content="1314454800" />
		<meta name="Copyright" content="latribune.fr" />
		<meta name="Language" content="fr" />
		<meta name="description" lang="fr" content="LaTribune.fr - Journal Quotidien boursier, économique et financier - Actualité et information bourse, finance et économie" />
		<meta name="Keywords" content="LaTribune.fr; Journal; Quotidien; boursier; économique; financier; Actualité; information; bourse; finance; économie; " />
		<meta http-equiv="pragma" content="no-cache" />
		<meta http-equiv="expires" content="0

In [12]:
requests.get(mirrors[2]).text

'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n<meta http-equiv="pragma" content="no-cache" />\r\n<meta http-equiv="cache-control" content="no-cache" />\r\n<meta http-equiv="expires" content="0" />\r\n<meta http-equiv="keywords" content="" />\r\n<meta http-equiv="description" content="HSIN Login" />\r\n<title>HSIN &gt; Login</title>\r\n<link href="css/hsinstyle.css" rel="stylesheet" type="text/css" />\r\n<script language="javascript" src="javascript/hsinscript.js"></script>\r\n<style type="text/css">\r\nbody {\r\n\tmargin-left: 0px;\r\n\tmargin-top: 0px;\r\n\tmargin-right: 0px;\r\n\tmargin-bottom: 0px;\r\n\tbackground-color: #003366;\r\n\tfont-size:10pt;\r\n\tpadding:0;\r\n\tmargin:0 auto;\r\n}\r\nbody, h1, h2 {font-family:Arial;}\r\nh1 {font-size:1.2em;}\r\n/*Footer*/\r\n.foo